In [1]:
#Sentiment Analysis
#1Preparation
#2datacleaning dan preprocessing
#3 feature extraction

In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
cols=['sentiment','id','date','query_string','user','text']
df_training=pd.read_csv("C:/Users/Ichida/Documents/Digitalent/trainingdata.csv",encoding="ISO-8859-1",header=None,names=cols)
df_training.head()

,sentiment,id,date,query_string,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [11]:
df_training.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600000 entries, 0 to 1599999
Data columns (total 6 columns):
sentiment       1600000 non-null int64
id              1600000 non-null int64
date            1600000 non-null object
query_string    1600000 non-null object
user            1600000 non-null object
text            1600000 non-null object
dtypes: int64(2), object(4)
memory usage: 73.2+ MB


In [12]:
df_training.sentiment.value_counts()

4    800000
0    800000
Name: sentiment, dtype: int64

In [13]:
df_training['sentiment']=df_training['sentiment'].map({0:0,4:1})

In [14]:
df_training.sentiment.value_counts()

1    800000
0    800000
Name: sentiment, dtype: int64

In [15]:
#data cleaning and preprocessing
from bs4 import BeautifulSoup
import re
from nltk.tokenize import WordPunctTokenizer
from nltk.corpus import stopwords
tok = WordPunctTokenizer()
combined_pat=r'|'.join((pat1,pat2))
stop_words=set(stopwords.words('english'))

In [18]:
def tweet_cleaner(text):
    soup=BeautifulSoup(text,'html.parser')
    souped=soup.get_text()
    stripped=re.sub(combined_pat,'',souped)
    try:
        clean=stripped.decode("utf-8").replace(u"\ufffd","?")
    except:
        clean=stripped
    letters_only=re.sub("[^a-zA-Z]","",clean)
    lower_case=letters_only.lower()
    words=tok.tokenize(lower_case)
    filtered_words=[w for w in words if not w in stop_words]
    return("".join(filtered_words)).strip()

In [24]:
%%time
print("cleaning and parsing the tweets... \n")
clean_tweet_text=[]
for i in range(len(df_training)):
    clean_tweet_text.append(tweet_cleaner(df_training['text'][i]))
print("Done")
clean_df = pd.DataFrame(clean_tweet_text,columns=['text'])
clean_df['target']=df_training.sentiment
clean_df.head()

cleaning and parsing the tweets... 

Done
Wall time: 7min 39s


In [27]:
clean_df.to_csv('clean_tweet_training.csv',encoding='utf-8')

NameError: name 'clean_df' is not defined

In [ ]:
csv='clean_tweet_training.csv'
my_df=pd.read_csv(csv,index_col=0)
my_df.head()

In [ ]:
tweet_text=my_df['text']
target= my_df['target']
fixed_text=tweet_text[pd.notnull(tweet_text)]
fixed_target=target[pd.notnull(tweet_text)]

In [ ]:
# Feature Extraction
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(fixed_text)
X